# Automatic Speech Recognition --- GMM, HMM

### Main steps:
1. Extract features by MFCC (Mel-frequency cepstral coefficients).
2. Calculate the distribution of features for a phone by a Gaussian Mixture Model (GMM).
3. Calculate the transition between phones and the corresponding Observable by Hidden Markov Model (HMM).
4. Find the optimal sequence by Viterbi decoder.

### Hidden Markov Model 

A Markov chain contains all the possible states of a system and the probability of transiting from one state to another. Each state transition only depends on the previous N states. This process is called an n-order model, where n is the number of states that affect the transition. The simplest Markov process is a first-order process, and the transition of each state only depends on the previous state. It is expressed in mathematical expression as follows:

$$P(X_{n+1}=x|X_1=x_1,X_2=x_2,...,X_n=x_n)=p(X_{n+1}=x|X_n=x_n)$$

However, this model leads to the lack of information. Take coin tossing as an example. We toss a coin for several times and we can see the outcomes (Heads and Tails). In general, the probabilities of Head or Tail are 0.5. But in this case we don't know whether the coin is fair, maybe the probability of Head is 0.3 and the Tail is 0.7. Therefore, we need to introduce Hidden Markov Model (HMM) to solve this problem. Assume we have two coins, one is fair and the other is unfair. So we can get a state set: {fair, biased}. And we don't know the current coin belongs to which type, but we can infer it from what we observe. For instance, when the coin is fair, the probabilities that we can get Head and Tail are both 0.5, but when the coin is biased, we only have 0.3 chance to get Head and 0.7 chance to get Tail. To describe this situation, we can introduce two terminologies. Emission probability represents the probability that we observe an observable object given an hidden state. And the transition probability represents the probability of transiting from one hidden state to another (change from fair coin to biased coin). 

I am already finished an HMM model, so let's take a look.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# show all the columns
pd.set_option('display.max_columns',None)

# set pi, transition_probility, emission_probility (in reality, these parameters will get from GMM)
pi = np.array([0.5,0.5])
transition_probability = np.array([[0.9,0.1],[0.1,0.9]])
emission_probability = np.array([[0.5,0.5],[0.75,0.25]])

# observable sequence
obs_seq_with_labels = ['Head','Head','Head','Head','Head','Tail','Tail','Tail','Tail','Tail']
obs_seq = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

# two state of coin
states = ['Fair','Biased']

# in order to show clearly, add labels to data
def preprocess_data(m_data=None,m_index=None,m_columns=None):
    return pd.DataFrame(data=m_data,index=m_index,columns=m_columns)

# add labels to data
pi_with_labels = preprocess_data(m_data=pi,m_index=states,m_columns=['Start'])
transition_probability_with_labels = preprocess_data(m_data= transition_probability,m_index=states,m_columns=states)
emission_probability_with_labels = preprocess_data(m_data=emission_probability,m_index=states,m_columns=['Head','Tail'])

# show data
print('sequence of coin:')
print(obs_seq_with_labels)
print()
print('initial statues:')
print(pi_with_labels)
print() 
print('transition probability:')
print(transition_probability_with_labels)
print()
print('emission probability:')
print(emission_probability_with_labels)
print()

sequence of coin:
['Head', 'Head', 'Head', 'Head', 'Head', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail']

initial statues:
        Start
Fair      0.5
Biased    0.5

transition probability:
        Fair  Biased
Fair     0.9     0.1
Biased   0.1     0.9

emission probability:
        Head  Tail
Fair    0.50  0.50
Biased  0.75  0.25



In order to build a HMM and show the functions of the HMM clearly, I set these data initially.

#### Forward Algorithm

When a HMM model is learned, we can get all the data above. In this case, we can use Forward Algorithm to calculate the probability of every observation. And in this step, the goal is to sum the probabilities of the observations for all possible state sequences:

$$f_t(i+1) = e_t(y_{i+1})\sum_{s}f_s(i)p_{s,t}$$

In this formula, e represents emission probability and p represents transition probability. It shows clearly that the probability of current observation is equal to the sum of the last observation for its possible states. However, if we sum over all possible state sequences one-at-a-time. It has exponential complexity. Therefore, in order to improve the efficiency, we need to use Dynamic Programming (DP) to store the results we already get. By using DP, we can avoid repeating calculation. Following is the function of Forward Algorithm I implemented.

In [3]:
def calculate_forward_probabilities(pi, emission_probability, transition_probability, obs_seq):
    row = transition_probability.shape[0]
    col = len(obs_seq)
    F = np.zeros((row, col))

    # calculate the first probability of observation
    F[:, 0] = pi * emission_probability[:, obs_seq[0]]

    # calculate other probabilities by the formula above
    for c in range(1, col):
        for r in range(row):
            F[r, c] = sum(F[:, c - 1] * np.transpose(transition_probability[r, :]))
        F[:, c] = F[:, c] * emission_probability[:, obs_seq[c]]
    return F


forward_matrix = calculate_forward_probabilities(pi, emission_probability, transition_probability, obs_seq)

# add labels to data
forward_matrix_with_labels = preprocess_data(m_data=forward_matrix,m_columns=obs_seq_with_labels,m_index=states)

# show the results
print(forward_matrix_with_labels)

         Head      Head      Head      Head      Head      Tail      Tail  \
Fair    0.250  0.131250  0.072656  0.042363  0.025862  0.016386  0.008474   
Biased  0.375  0.271875  0.193359  0.135967  0.094955  0.022011  0.005362   

            Tail      Tail      Tail  
Fair    0.004081  0.001908  0.000879  
Biased  0.001418  0.000421  0.000142  


Look at the outcomes above, the first row represents the observable objects, the second and third row represent the probabilities of two state of coins respectively.

#### Viterbi Algorithm

The next step, we need to find the hidden states by given a observable objects sequence. In other words, given an audio clip, we can get the words. And this process is called decoding. Suppose the probability $v_s(i)$ of the most probable path ending in state s at position i for y is known for all the states s, then we can compute $v_t(i+1)$ as follows:

$$v_t(i+1) = e_t(y_{i+1})*max_s(v_s(i)p_{s,t})$$

This formula is quite similar to Forward Algorithm, excepte that we need to implement max operation here rather than sum operation. Also, in order to avoid repeating calculation, we need to use DP method to store the previous results. The objective of Viterbi algorithm is to find the most likely path, in other words, we need to find a sequence of hidden states that maximize the likelihood of observable objects. Therefore, during this process, we also need to store each node of the maximum path. And then, we backtrack the optimal path for each node. Following is the function of Forward Algorithm I implemented.

In [4]:
viterbi_path = []

def viterbi(pi, emission_probability, transition_probability, obs_seq):
    row = transition_probability.shape[0]
    col = len(obs_seq)
    F = np.zeros((row, col))

    # calculate the first probability of observation
    F[:, 0] = pi * emission_probability[:, obs_seq[0]]
    t = pd.DataFrame(F[:, 0])
    viterbi_path.append(t.idxmax()[0])

    # calculate other probabilities by the formula above
    for c in range(1, col):
        for r in range(row):
            F[r, c] = max(np.transpose(transition_probability[r, :]) * F[:, c - 1])
        F[:, c] = F[:, c] * emission_probability[:, obs_seq[c]]
        t = pd.DataFrame(F[:, c])

        # store the nodes
        viterbi_path.append(t.idxmax()[0])

    return F


viterbi_matrix = viterbi(pi, emission_probability, transition_probability, obs_seq)

# add labels to data
viterbi_matrix_with_labels = preprocess_data(m_data=viterbi_matrix,m_columns=obs_seq_with_labels,m_index=states)
print(viterbi_path)
print()

# back track
def calculate_viterbi_path():
    prev_state = viterbi_path[-1]

    for i in range(len(viterbi_path)):
        viterbi_path[i] = states[viterbi_path[i]]

calculate_viterbi_path()

# show results
print('viterbi matrix:')
print(viterbi_matrix_with_labels)
print()
print('best path of viterbi matrix:')
print(viterbi_path)

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]

viterbi matrix:
         Head      Head      Head      Head      Head      Tail      Tail  \
Fair    0.250  0.112500  0.050625  0.022781  0.010252  0.004613  0.002076   
Biased  0.375  0.253125  0.170859  0.115330  0.077848  0.017516  0.003941   

            Tail     Tail      Tail  
Fair    0.000934  0.00042  0.000189  
Biased  0.000887  0.00020  0.000045  

best path of viterbi matrix:
['Biased', 'Biased', 'Biased', 'Biased', 'Biased', 'Biased', 'Biased', 'Fair', 'Fair', 'Fair']


We can observe the max probability of each observable object from viterbi matrix. And we can see the optimal path above.

#### Backward Algorithm

Before further discussion, let me introduce backward algorithm. This algorithm is similar to the Forward Algorithm but in the reverse direction.

$$b_s(i) = \sum_t b_t(i+1) e_t(y_{i+1})p_{s,t}$$

In [5]:
def calculate_backward_probabilities(emission_probability, transition_probability, obs_seq):
    row = transition_probability.shape[0]
    col = len(obs_seq)
    F = np.zeros((row, col))

    # calculate the last probability of observation
    F[:, -1] = np.ones((row))

    # calculate other probabilities by the formula above
    for c in range(col - 2, -1, -1):
        for r in range(row):
            t = np.array((F[r, c + 1], F[r, c + 1]))
            F[r, c] = (t * np.transpose(transition_probability[r, :]) * emission_probability[:, obs_seq[c]]).sum()
    return F


backward_matrix = calculate_backward_probabilities(emission_probability, transition_probability, obs_seq)

# add labels to data
backward_matrix_with_labels = preprocess_data(m_data=backward_matrix,m_columns=obs_seq_with_labels,m_index=states)

# show the results
print('backward algorithm:')
print(backward_matrix_with_labels)

backward algorithm:
            Head      Head      Head      Head      Head      Tail      Tail  \
Fair    0.002030  0.003867  0.007366  0.014031  0.026726  0.050907  0.107172   
Biased  0.001146  0.001580  0.002179  0.003006  0.004146  0.005719  0.020797   

            Tail   Tail  Tail  
Fair    0.225625  0.475   1.0  
Biased  0.075625  0.275   1.0  


The result above is backward matrix.

#### Baum–Welch Algorithm

Now, we have a HMM model and we need to train it. In other words, this is the time to get emission probability and transition probability. And Baum-Welch algorithm can help us to do it. Baum-Welch algorithm is actually an EM algorithm which contains two main steps: E-step and M-step. Generally, EM algorithm will implements those two steps iteratively. But before implementing Baum-Welch algorithm, we should be familiar with four notations. They are $\alpha_i(t)$, $\beta_i(t)$, $\gamma_i(t)$ and $\xi_{ij}(t)$. $\alpha$ represents Forward Algorithm and $\beta$ represents Backward Algorithm. $\gamma$ represents state occupation probability which means the probability of state i at time t given all the observed objects.

$$\gamma_i(t) = \frac{\alpha_i(t)\beta_i(t)}{\sum_{j=1}^N \alpha_j(t)\beta_j(t)}$$

Here's the code

In [6]:
def calculate_gamma(alpha, beta, obs_seq, transition_probability):
    row = transition_probability.shape[0]
    col = len(obs_seq)
    gamma = np.zeros((row, col), np.float)

    # calculate other probabilities by the formula above
    for t in range(col):
        for i in range(row):
            gamma[i, t] = alpha[i, t] * beta[i, t] / sum(alpha[j, t] * beta[j, t] for j in range(row))
    return gamma


gamma = calculate_gamma(forward_matrix, backward_matrix, obs_seq, transition_probability)

print('gamma:')
print(gamma)

gamma:
[[0.5416142  0.5416142  0.55947794 0.59254572 0.63709269 0.86887127
  0.89063733 0.89567305 0.88666354 0.86060367]
 [0.4583858  0.4583858  0.44052206 0.40745428 0.36290731 0.13112873
  0.10936267 0.10432695 0.11333646 0.13939633]]


The last one is $\xi$, $\xi$ represents the probability of transiting from state i to j after time t given all the oberved objects.

$$\xi_{ij}(t) = \frac{\alpha_i(t)a_{ij}\beta_j(t+1)b_j(y_{t+1})}{\sum_{i=1}^N\sum_{j=1}^N\alpha_i(t)a_{ij}\beta_j(t+1)b_j(y_{t+1})}$$

In this formula, the $a$ represents transition_probility and the $b$ represents emission_probility. Following is the function of $\xi$ I implemented.

In [7]:
def compute_xi(alpha, beta, obs_seq, emission_probability, transition_probability):
    col = len(obs_seq)
    row = transition_probability.shape[0]
    xi = np.zeros((row, row, col), np.float)

    # calculate other probabilities by the formula above
    for t in range(col - 1):
        for i in range(row):
            for j in range(row):
                numerator = alpha[i, t] * transition_probability[i, j] * emission_probability[j, obs_seq[t + 1]] * beta[
                    j, t + 1]
                denominator = sum(sum(
                    alpha[i1, t] * transition_probability[i1, j1] * emission_probability[j1, obs_seq[t + 1]] * beta[
                        j1, t + 1]
                    for j1 in range(row)) for i1 in range(row))

                xi[i, j, t] = numerator / denominator
    return xi


xi = compute_xi(forward_matrix, backward_matrix, obs_seq, emission_probability, transition_probability)
print('xi:')
print(xi)

xi:
[[[0.46424074 0.45480142 0.45731744 0.46961905 0.61711445 0.77496589
   0.83683627 0.85370016 0.83999704 0.        ]
  [0.03161281 0.02242658 0.01632978 0.0121431  0.00385167 0.00835463
   0.01558282 0.02745819 0.0466665  0.        ]]

 [[0.07737346 0.10467652 0.13522828 0.16747364 0.25175683 0.11567144
   0.05883678 0.03296338 0.02060663 0.        ]
  [0.42677299 0.41809549 0.3911245  0.35076421 0.12727705 0.10100803
   0.08874412 0.08587827 0.09272983 0.        ]]]


Now, we can start Baum–Welch Algorithm. In a nutshell, Baum-Welch Algorithm allows us to fix a pair of parameters to improve others and continue the iteration until the solution converges. And We can use a formula to represents it:

$$\theta_1^* =  \arg\max_{\theta_1}\sum_{x \in D}E_{\theta_2}logp(x, \theta_2;\theta_1)$$

In this formula, $\theta_1$ represents emission probability and transition probability, so M step is to find the optimal $\theta_1$ (emission probability and transition probability) given $\theta_2$ and x. And $\theta_2$ represents $\gamma$ and $\xi$. Therefore, E step means to establish the probability distribution of $\theta_2$ ($\gamma$ and $\xi$) given $\theta_1$ and x (x represents observation sequence). In previous part of this report, I already introduce how to calculate $\alpha$, $\beta$, $\gamma$ and $\xi$. Next, I will introduce how to evaluate emission probability and transition probability. Let me use a to represent transition_probability:

$$a_{ij}^* = \frac{\sum_{t=1}^{T-1}\xi_{ij}(t)}{\sum_{t=1}^{T-1}\gamma_i(t)}$$

Using b to represents emission probability:

$$b_i^*(v_k) = \frac{\sum_{t=1}^T 1_{y_t = v_k }\gamma_i(t)}{\sum_{t=1}^T \gamma_i(t)}$$

In this formula, v represents all of the states (in this case, {biased, fair}). So, the numerator of this formula means sum $\gamma$ over all time steps where the observation sequence $y_t$ is equal to $v_t$ at time t. t also can be seen as indexes of observation sequence.

In [51]:
def baum_welch(pi, emission_probability, transition_probability, obs_seq):
    
    # number of states
    M = emission_probability.shape[1]
    row = transition_probability.shape[0]
   
    col = len(obs_seq)
    
    # states
    V = [k for k in range(M)]

    # set threshold
    x = 0.2
    delta_lambda = x + 1
    
    # number of iterations
    times = 0
  
    while delta_lambda > x:  
        
        # E step: calculate alpha, beta, gamma and xi by functions I implemented above
        alpha = calculate_forward_probabilities(pi, emission_probability, transition_probability, obs_seq) 
        beta = calculate_backward_probabilities(emission_probability, transition_probability, obs_seq) 
        gamma = calculate_gamma(alpha, beta, obs_seq, transition_probability)
        xi = compute_xi(alpha, beta, obs_seq, emission_probability, transition_probability)

        # store the last results
        lambda_n = [transition_probability.copy(), emission_probability.copy(), pi.copy()]

        # M step: calculate transition probability and emission probability by the formulas I mentioned above
        for i in range(row):
            for j in range(row):
                numerator = sum(xi[i, j, t] for t in range(col - 1))
                denominator = sum(gamma[i, t] for t in range(col - 1))
                transition_probability[i, j] = numerator / denominator

        for j in range(row):
            for k in range(M):
                numerator = sum(gamma[j, t] for t in range(col) if obs_seq[t] == V[k]) 
                denominator = sum(gamma[j, t] for t in range(col))
                emission_probability[j, k] = numerator / denominator

            
        # calculate offset to judge whether the values converge
        delta_transition_probability = map(abs, lambda_n[0] - transition_probability) 
        delta_emission_probability = map(abs, lambda_n[1] - emission_probability)
        delta_pi = map(abs, lambda_n[2] - pi)
        delta_lambda = sum([sum(sum(delta_transition_probability)), sum(sum(delta_emission_probability)), sum(delta_pi)])
#        print(delta_lambda)
#        times += 1
#        print(times)

    return transition_probability, emission_probability, pi

In [52]:
# set transition probability and emission probability casually.
transition_probability = np.array([[0.8, 0.2], [0.2, 0.8]], np.float)
emission_probability = np.array([[0.5, 0.5], [0.75, 0.25]], np.float)

# get results
transition_probability, emission_probability, pi = baum_welch(pi, emission_probability, transition_probability, obs_seq)

# show results
print('transition_probability:')
print(transition_probability)
print()
print('emission_probability:')
print(emission_probability)


transition_probability:
[[8.37821393e-01 5.15828008e-04]
 [3.47724687e-01 7.68068811e-01]]

emission_probability:
[[1.15592651e-05 9.99988441e-01]
 [9.47473716e-01 5.25262842e-02]]


As we can see, we can use an observation sequence to train the model to get the transition probability and emission probability. Then we can use the transition probability, emission probability, observation sequence and the Viterbi Algorithm to find the optimal word sequence.